In [1]:
using Primes
using Primes: wheel, wheel_index, wheel_prime

function _primesmask_BitVector(limit::Int)
    limit < 7 && throw(ArgumentError("The condition limit ≥ 7 must be met."))
    n = wheel_index(limit)
    m = wheel_prime(n)
    sieve = trues(n)
    @inbounds for i = 1:wheel_index(isqrt(limit))
        if sieve[i]
            p = wheel_prime(i)
            q = p^2
            j = (i - 1) & 7 + 1
            while q ≤ m
                sieve[wheel_index(q)] = false
                q += wheel[j] * p
                j = j & 7 + 1
            end
        end
    end
    return sieve
end

function my_primes(n)
    list = [2, 3, 5]
    lo, hi = 2, n
    # http://projecteuclid.org/euclid.rmjm/1181070157
    sizehint!(list, 5 + floor(Int, hi / (log(hi) - 1.12) - lo / (log(lo) - 1.12 * (lo > 7))))
    sieve = _primesmask_BitVector(n)
    @inbounds for i = 1:length(sieve)   # don't use eachindex here
        sieve[i] && push!(list, wheel_prime(i))
    end
    list
end

@show _primesmask_BitVector(100)
@show my_primes(100);

_primesmask_BitVector(100) = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1]
my_primes(100) = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [2]:
using BenchmarkTools

In [3]:
P = @btime my_primes(10^5);

  215.700 μs (4 allocations: 78.80 KiB)


In [4]:
Q = @btime primes(10^5);

  173.800 μs (5 allocations: 101.50 KiB)


In [5]:
P == Q

true

In [6]:
PP = @time primes(10^10)

 42.988508 seconds (5 allocations: 5.885 GiB, 0.03% gc time)


455052511-element Vector{Int64}:
          2
          3
          5
          7
         11
         13
         17
         19
         23
         29
         31
         37
         41
          ⋮
 9999999707
 9999999727
 9999999769
 9999999781
 9999999787
 9999999817
 9999999833
 9999999851
 9999999881
 9999999929
 9999999943
 9999999967

In [7]:
function naive_primes(N)
    P = typeof(N)[]
    for n in 2:N
        for i in 2:isqrt(n)
            n % i == 0 && @goto next
        end
        push!(P, n)
        @label next
    end
    P
end

naive_primes (generic function with 1 method)

In [8]:
naive_primes(100) == primes(100)

true

In [9]:
@btime naive_primes(10^5)

  6.586 ms (9 allocations: 326.55 KiB)


9592-element Vector{Int64}:
     2
     3
     5
     7
    11
    13
    17
    19
    23
    29
    31
    37
    41
     ⋮
 99859
 99871
 99877
 99881
 99901
 99907
 99923
 99929
 99961
 99971
 99989
 99991

In [10]:
naive_primes(10^5) == primes(10^5)

true

In [11]:
function naive_first_primes(K)
    P = typeof(K)[]
    k = 0
    n = 2
    while k < K
        for i in 2:isqrt(n)
            n % i == 0 && @goto next
        end
        push!(P, n)
        k += 1
        @label next
        n += 1
    end
    P
end

naive_first_primes (generic function with 1 method)

In [12]:
using BenchmarkTools

In [13]:
P = @btime naive_first_primes(10^5)

  222.279 ms (11 allocations: 1.83 MiB)


100000-element Vector{Int64}:
       2
       3
       5
       7
      11
      13
      17
      19
      23
      29
      31
      37
      41
       ⋮
 1299533
 1299541
 1299553
 1299583
 1299601
 1299631
 1299637
 1299647
 1299653
 1299673
 1299689
 1299709

In [14]:
using Primes
Q = @btime nextprimes(2, 10^5)
P == Q

  119.658 ms (201896 allocations: 3.84 MiB)


true